# VIDUS imputation
__Author__: Jesse Marks <br>

**GitHub Issue**: [#42](https://github.com/RTIInternational/bioinformatics/issues/42)

This document logs the steps taken to perform pre-imputation procedures on the [Vancouver Injection Drug Users Study (VIDUS)](https://www.bccsu.ca/vidus/) dataset, EAs. The starting point for this analysis is after quality control of observed genotypes. The quality controlled genotypes are oriented on the GRCh37 plus strand. 

## Software and tools
The software and tools used for porcessing these data are
* [Michigan Imputation Server](https://imputationserver.sph.umich.edu/index.html) (MIS)
* [Amazon Web Services (AWS) - Cloud Computing Services](https://aws.amazon.com/)
    * Linux AMI
* [PLINK v1.90 beta 4.10](https://www.cog-genomics.org/plink/)
* [bgzip](http://www.htslib.org/doc/tabix.html)
* [BCF Tools](http://www.htslib.org/doc/bcftools.html)
* Windows 10 with [Cygwin](https://cygwin.com/) installed
* GNU bash version 4.2.46

## Data retrieval and organization
PLINK binary filesets will be obtained from AWS S3 storage.

Jesse Marks performed the QC and stored the observed genotype data at:

```
s3/rti-hiv/vidus/data/genotype/observed/final/
```

## Create Directory Structure & Download Data
The following section needs to be modified each time to reflect where the data is stored!

**Note**: download genotype data for each study to respective data directories. For example, download STUDY genotype data (post-QC) to the appropriate directory: `${base_dir}/processing/STUDY/genotype/observed/$ancestry`

In [1]:
### EC2 command line (bash) ###

# Modify variables below
######################################################################
base_dir=/shared/jmarks/heroin/vidus/genotype/impute/processing
genD=/shared/jmarks/heroin/vidus/genotype/observed/final

base_name="chr_all" # chr23 or chr_all
chr_list={1..23} # {1..23} or {1..22} 
ancestry_list="ea" # space delimited Ex. "ea aa ha"
study_list="vidus" # space delimited, lowercase
######################################################################

mkdir -p ${base_dir}/../final
mkdir ${base_dir}/{intersect,1000g,impute_prep} 
for study in ${study_list};do
    for ancestry in ${ancestry_list};do
        mkdir -p ${base_dir}/${study}/genotype/observed/${ancestry}
    done
done

## download genotype (with AWS CLI tools) to respective directories ##

# cd /shared/jmarks/heroin/vidus/genotype/impute/processing/vidus/genotype/observed/ea
# ln -s /shared/jmarks/heroin/vidus/genotype/observed/final/ea_chr_all.* .

# Data Processing
## GRCh37 strand and allele discordance check
### MAF for study data

In [ ]:
### EC1 command line (Bash) ###

# write out the MAF report
for study in ${study_list}; do
    study_dir=${base_dir}/${study}/strand_check
    mkdir ${study_dir}
    for ancestry in ${ancestry_list};do
        data_dir=${base_dir}/${study}/genotype/observed/${ancestry}
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed ${data_dir}/*bed\
            --bim ${data_dir}/*bim\
            --fam ${data_dir}/*fam\
            --freq \
            --out ${study_dir}/${ancestry}_${base_name}
    done
done

# Get list of variants from all studies
for ancestry in ${ancestry_list}; do
    for study in ${study_list};do
        cat ${base_dir}/${study}/genotype/observed/${ancestry}/*bim | \
                perl -lane 'if (($F[0]+0) <= 23) { print $F[1]; }' | \
                sort -u > ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt
    done
done

```
wc -l ${base_dir}/*txt
686744 /shared/jmarks/heroin/vidus/genotype/impute/processing/ea_chr_all_sorted_variants.txt
```

### MAF for 1000G
This pipeline is currently set up to handle EUR and AFR populations.
#### Autosomes
Get 1000G MAF for chromosomes 1–22 (autosomes).

In [ ]:
### EC2 command line (Bash)

# Calculate autosome MAFs for 1000G populations
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
    for chr in {1..22}; do
        /shared/bioinformatics/software/scripts/qsub_job.sh \
            --job_name ${pop}_${chr}_MAF \
            --script_prefix ${base_dir}/1000g/${pop}_chr${chr}.maf \
            --mem 6.8 \
            --nslots 1 \
            --priority 0 \
            --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
                --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.hap.gz\
                --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.legend.gz \
                --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
                --chr ${chr} \
                --out ${base_dir}/1000g/${pop}_chr${chr}.maf \
                --extract ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt \
                --keep_groups ${pop}
    done
done

#### chrX
Get 1000G MAF for chromosome 23 (chrX).

In [ ]:
### Bash ###

chr=23
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi

    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${pop}_23_MAF \
        --script_prefix ${base_dir}/1000g/${pop}_chr${chr}.maf \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
            --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.hap.gz\
            --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.legend.gz \
            --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
            --chr $chr \
            --out ${base_dir}/1000g/${pop}_chr${chr}.maf \
            --extract ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt \
            --keep_groups ${pop}
done

### Merge 1000G chromosomes
Only need to perform this if there were multiple chromosomes for which the MAF was calculated—e.g. more than just chrX.

In [ ]:
### Bash ###

# Merge per chr MAFs for each 1000G population
for ancestry in ${ancestry_list};do
    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
        head -n 1 ${base_dir}/1000g/${pop}_chr1.maf > ${base_dir}/1000g/${pop}_chr_all.maf
        tail -q -n +2 ${base_dir}/1000g/${pop}_chr??maf  >> ${base_dir}/1000g/${pop}_chr_all.maf
done

```
wc -l ${base_dir}/1000g/???_chr_all.maf
385648 /shared/jmarks/heroin/vidus/genotype/impute/processing/1000g/EUR_chr_all.maf
```

###  Allele Discordances Check
The allele discordances will be resolved by
* Flipping allele discordances that are fixed by flipping
* Removing SNPs with discordant names
* Removing SNPs with discordant positions
* Removing allele discordances that are not resolved by flipping
* Removing alleles with large deviations from the reference population allele frequencies

Given that the allele discordance check was done using a union set of SNPs across all studies within an ancestry group, some of the SNPs logged as discordant for a given study may not actually be in the study. Fortunately, if they are not in a given study they will not interfere with the filtering procedures. Note that the intersection set is used for the final studies merger.

#### Autosomes

In [ ]:
### Bash ###

chr="all"
# Run discordance checks for each ancestry group
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        if [ $ancestry = "ea" ]; then
            pop=EUR
        else
            pop=AFR
        fi

       /shared/bioinformatics/software/scripts/qsub_job.sh \
           --job_name ${ancestry}_${study}_crosscheck_chr_${chr} \
           --script_prefix ${base_dir}/$study/strand_check/${ancestry}_allele_discordance_check \
           --mem 6 \
           --nslots 4 \
           --priority 0 \
           --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
               --study_bim_file ${base_dir}/${study}/genotype/observed/${ancestry}/*bim
               --study_frq_file ${base_dir}/${study}/strand_check/${ancestry}_chr_all.frq
               --ref_maf_file ${base_dir}/1000g/${pop}_chr_all.maf
               --out_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance"
    done
done

#### chrX 
The following code should not be run unless chr23 was the only chromosome processed.

In [ ]:
#### Bash ###
#
#chr=23
#for study in ${study_list}; do
#    for ancestry in ${ancestry_list};do
#        if [ $ancestry = "ea" ]; then
#            pop=EUR
#        else
#            pop=AFR
#        fi
#
#        # chr23 discordance check
#        /shared/bioinformatics/software/scripts/qsub_job.sh \
#            --job_name ${ancestry}_${study}_crosscheck_chr${chr} \
#            --script_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance_check \
#            --mem 6.8 \
#            --nslots 1 \
#            --priority 0 \
#            --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
#                --study_bim_file ${base_dir}/${study}/genotype/observed/${ancestry}/*bim
#                --study_frq_file ${base_dir}/${study}/strand_check/${ancestry}_chr23.frq
#                --ref_maf_file ${base_dir}/1000g/${pop}_chr23.maf
#                --out_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance_${chr}"
#    done
#done

### Resolving Allele Discordances

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        echo -e "\n===============\nProcessing ${study}_${ancestry}\n"
        echo "Making remove list"
        cat <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.at_cg_snps_freq_diff_gt_0.2 | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_names | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_positions | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_polymorphic_in_study_not_fixed_by_strand_flip | tail -n +2) | \
              sort -u > ${base_dir}/${study}/strand_check/${ancestry}_snps.remove

        # Create flip list
        echo "Making flip list"
        comm -23 <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles | tail -n +2 | sort -u) \
                 <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2 | sort -u) \
                 > ${base_dir}/${study}/strand_check/${ancestry}_snps.flip

        # Apply filters
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed     ${base_dir}/${study}/genotype/observed/${ancestry}/*bed \
            --bim     ${base_dir}/${study}/genotype/observed/${ancestry}/*bim \
            --fam     ${base_dir}/${study}/genotype/observed/${ancestry}/*fam \
            --exclude ${base_dir}/${study}/strand_check/${ancestry}_snps.remove \
            --flip    ${base_dir}/${study}/strand_check/${ancestry}_snps.flip \
            --make-bed \
            --out     ${base_dir}/${study}/${ancestry}_filtered
    done
done

 ```
 wc -l ${base_dir}/*/*bim
682845 /shared/jmarks/heroin/vidus/genotype/impute/processing/vidus/ea_filtered.bim
```

## Remove monomorphic variants
Monomorphic variants prevent MIS from accepting the genotype data. In this case, an arbitrarily small MAF is set that is smaller than the lower bound for these data.

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        geno_dir=${base_dir}/${study}

        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bfile ${geno_dir}/${ancestry}_filtered \
            --maf 0.000001 \
            --make-bed \
            --out ${geno_dir}/${ancestry}_filtered_mono
    done
done

```
 wc -l ${base_dir}/*/*mono.bim
653507 /shared/jmarks/heroin/vidus/genotype/impute/processing/vidus/ea_filtered_mono.bim
```

### SNP intersection
Only run if merging multiple data sets.

In [ ]:
#studies=($study_list)  #studies=(UHS1 UHS2 UHS3_v1-2 UHS3_v1-3) # array of study names
#num=${#studies[@]}
#
## Get intersection set
#for ancestry in ${ancestry_list};do
#    bim_files=()
#    for (( i=0; i<${num}; i++ ));do
#        bim_files+=(${base_dir}/processing/${studies[$i]}/${ancestry}_filtered_mono.bim)
#    done
#    
#    echo -e "\nCalculating intersection between $ancestry ${study_list}...\n"
#    sort ${bim_files[@]} | uniq -dc | awk -v num=$num '$1 == num {print $3}' \
#        > ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt
#
#    # Make new PLINK binary file sets
#    for study in ${studies[@]}; do
#        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#            --noweb \
#            --bfile ${base_dir}/processing/${study}/${ancestry}_filtered_mono \
#            --extract ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt \
#            --make-bed \
#            --out ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection
#    done
#done

```wc -l *txt

```

### Merge test
If merging multiple datasets together:

As a final check to confirm that our data sets are all compatible, a PLINK file set merge is conducted. If any issues persist then an error will be raised.

In [ ]:
#for ancestry in $ancestry_list;do
#
#    echo "Creating $ancestry merge-list"
#    touch ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    for study in $study_list;do
#        echo ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection >>\
#             ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    done
#
## Merge file sets
#    echo -e "\n\n======== ${ancestry} ========\n\n"
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --merge-list ${base_dir}/processing/intersect/${ancestry}_merge_list.txt \
#        --make-bed \
#        --out ${base_dir}/processing/intersect/${ancestry}_studies_merged
#done

```
 wc -l *merged.bim
```

## Imputation preparation for Michigan Imputation Server
Visit the [MIS Getting Started Webpage](https://imputationserver.sph.umich.edu/start.html#!pages/help) for more information about the preparing the data for upload to MIS.


### VCF File Conversion

#### If merging was done

In [ ]:
### Split by chr and remove any individuals with missing data for whole chr

## if merged data sets together
#for ancestry in $ancestry_list;do
#    # Remove SNPs
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --bfile ${base_dir}/processing/intersect/${ancestry}_studies_merged \
#        --chr ${chr} \
#        --mind 0.99 \
#        --make-bed \
#        --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
#done > ${base_dir}/processing/impute_prep/chr_splitting.log 

#### If NO merging was performed 
I.e. if there was only one data set being processing.

In [ ]:
## if NO merging was done
for ancestry in $ancestry_list;do
    for chr in {1..23};do
        # Remove SNPs
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 4000 \
            --bfile ${base_dir}/$study/${ancestry}_filtered_mono \
            --chr ${chr} \
            --mind 0.99 \
            --make-bed \
            --out ${base_dir}/impute_prep/${ancestry}_chr${chr}_for_phasing 
    done
done > ${base_dir}/impute_prep/chr_splitting.log 

__Note__: No subjects were removed.

```
/shared/jmarks/heroin/vidus/genotype/impute/processing/impute_prep/ea_chr10_for_phasing.log:0 people removed due to missing genotype data (--mind).
/shared/jmarks/heroin/vidus/genotype/impute/processing/impute_prep/ea_chr11_for_phasing.log:0 people removed due to missing genotype data (--mind).

...

/shared/jmarks/heroin/vidus/genotype/impute/processing/impute_prep/ea_chr8_for_phasing.log:0 people removed due to missing genotype data (--mind).
/shared/jmarks/heroin/vidus/genotype/impute/processing/impute_prep/ea_chr9_for_phasing.log:0 people removed due to missing genotype data (--mind).```

In [ ]:
# EC2 command line #

for chr in {1..23};do
    for ancestry in ${ancestry_list};do
        final_dir=${base_dir}/impute_prep/${ancestry}
        mkdir $final_dir
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 5000 \
            --bfile ${final_dir}/../${ancestry}_chr${chr}_for_phasing \
            --output-chr M \
            --set-hh-missing \
            --recode vcf bgz \
            --out ${final_dir}/${ancestry}_chr${chr}_final
    done
done

Transfer the *.vcf.gz files to local machine (per chromosome) and then upload to MIS.

## Upload to Michigan Imputation Server (MIS)

### Uploading parameters
These are the parameters that were selected on MIS.

__Name__: VIDUS_EA

__Reference Panel__ 1000G Phase 3 v5

__Input Files__ File Upload <br>

* Select Files - select VCF (.gz) files that were downloaded to local machine from cloud. <br>

__Phasing__: ShapeIT v2.r790 (unphased) 

__Population__: EUR

__Mode__: Quality Control & Imputation

* I will not attempt to re-identify or contact research participants.
* I will report any inadvertent data release, security breach or other data management incident of which I become aware.

# Download Imputed Results from MIS
First, download the data form the Michigan Imputation Server (MIS) by clicking on the link provided in the email they send out to alert you that your data has finished. Here you will find commands for downloading the data.

## EA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* EA password: kyQ9D3h9PZWos

Then, inflate imputation results.

In [ ]:
ancestry=ea
passW=kyQ9D3h9PZWos
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/final/$ancestry

# QC-results
wget https://imputationserver.sph.umich.edu/share/results/ecbbfbb8dd22b1e62ce0d3d9c10ba2f8/qcreport.html

# SNP Statistics
wget https://imputationserver.sph.umich.edu/share/results/5a5b03861174bb36d72b966ca7c174a3/statistics.txt


# Logs
wget https://imputationserver.sph.umich.edu/share/results/a574d6d950193ece5bf3b67e29428909/chr_1.log
wget https://imputationserver.sph.umich.edu/share/results/95c9951e18635b35f7a99b4e951beb77/chr_10.log
wget https://imputationserver.sph.umich.edu/share/results/417d40ca0f151d0eac7825ba0b24d6df/chr_11.log
wget https://imputationserver.sph.umich.edu/share/results/7555fc95a13dfb787e54076be819f2af/chr_12.log
wget https://imputationserver.sph.umich.edu/share/results/8d589cf89f544e6ffee6db2b0378fa8d/chr_13.log
wget https://imputationserver.sph.umich.edu/share/results/46b93bce4701fbb3a436db23bf8e0ef6/chr_14.log
wget https://imputationserver.sph.umich.edu/share/results/387f5f2402bfa22ba7e3852a82d187eb/chr_15.log
wget https://imputationserver.sph.umich.edu/share/results/e5a1df7922fc32ff01c780fc19090673/chr_16.log
wget https://imputationserver.sph.umich.edu/share/results/43ea7776b28b295cbf8358ddad217e3e/chr_17.log
wget https://imputationserver.sph.umich.edu/share/results/188e8efffe4f81207d9cd16197d1e9ac/chr_18.log
wget https://imputationserver.sph.umich.edu/share/results/6c6002019a4fc5ca521a7f017554583b/chr_19.log
wget https://imputationserver.sph.umich.edu/share/results/a55502c3dbcf808ee95a671e1c04acb7/chr_2.log
wget https://imputationserver.sph.umich.edu/share/results/5226ebe22979a8cf14495f08d590a483/chr_20.log
wget https://imputationserver.sph.umich.edu/share/results/7033b4a41c3be9178b3e7606e44e281d/chr_21.log
wget https://imputationserver.sph.umich.edu/share/results/cb8794938f682520f76043e9e71bedde/chr_22.log
wget https://imputationserver.sph.umich.edu/share/results/720c068b2275322106e4af6559af621f/chr_3.log
wget https://imputationserver.sph.umich.edu/share/results/d3286d9bc5e7274f09f5e364c7cdd4a/chr_4.log
wget https://imputationserver.sph.umich.edu/share/results/9c542f85c0d4729caead6d95607c7455/chr_5.log
wget https://imputationserver.sph.umich.edu/share/results/f9a3707ff5e85ce8650074af46cee5ae/chr_6.log
wget https://imputationserver.sph.umich.edu/share/results/4f1679df0c84ecd3b7ff32ecddecf774/chr_7.log
wget https://imputationserver.sph.umich.edu/share/results/11c5bb44f35ae97a7cb7ae823e0e9b95/chr_8.log
wget https://imputationserver.sph.umich.edu/share/results/d2d388f00804f185a576a992656e3fa3/chr_9.log
wget https://imputationserver.sph.umich.edu/share/results/dbaea61a38c47d22039bea660dd86507/chr_X.no.auto_female.log


# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/d4d707d0b541259a379587abb643374f/chr_1.zip
wget https://imputationserver.sph.umich.edu/share/results/cbe570d8b2376a115e617bc812682e4f/chr_10.zip
wget https://imputationserver.sph.umich.edu/share/results/8ff9d002a80fc5dc0f900f8a2e21e346/chr_11.zip
wget https://imputationserver.sph.umich.edu/share/results/4d23b127de40f4288fa463a3ac0a1832/chr_12.zip
wget https://imputationserver.sph.umich.edu/share/results/d323fa78aa2988fd9563e66ca9a84bd5/chr_13.zip
wget https://imputationserver.sph.umich.edu/share/results/55323402c7d85372366ee1ca34375d9c/chr_14.zip
wget https://imputationserver.sph.umich.edu/share/results/e316e9fd7df9ea92da921685f188e6f5/chr_15.zip
wget https://imputationserver.sph.umich.edu/share/results/775e3e52ff664452a08c922c93018118/chr_16.zip
wget https://imputationserver.sph.umich.edu/share/results/f1ba42d0d3757e0193ab6c3eeaba04ea/chr_17.zip
wget https://imputationserver.sph.umich.edu/share/results/b85fe2f8236d68fa89fc2baa65827e2f/chr_18.zip
wget https://imputationserver.sph.umich.edu/share/results/c726fbe8eff6a225e73c1977b94ee1f2/chr_19.zip
wget https://imputationserver.sph.umich.edu/share/results/8d0bf2e50599de28080ca8a183bb283a/chr_2.zip
wget https://imputationserver.sph.umich.edu/share/results/37cef67fd24bf36c8d0eb024e248fc6d/chr_20.zip
wget https://imputationserver.sph.umich.edu/share/results/4b19df5b2ef8be81cccfbbc73850e59/chr_21.zip
wget https://imputationserver.sph.umich.edu/share/results/b73d2b4ef3d27cbffff6760bc117f441/chr_22.zip
wget https://imputationserver.sph.umich.edu/share/results/4513d0480f2bc5db76eb4fb70a6f20c4/chr_3.zip
wget https://imputationserver.sph.umich.edu/share/results/1ccac332a1e254e38ba76e36fee2be97/chr_4.zip
wget https://imputationserver.sph.umich.edu/share/results/99e36681a3cb633619a6fa9282688358/chr_5.zip
wget https://imputationserver.sph.umich.edu/share/results/be6747fcbe03f2bdd2f0aef31f89ee59/chr_6.zip
wget https://imputationserver.sph.umich.edu/share/results/ea3a1306509c98a0de9cb8eee8c3fcce/chr_7.zip
wget https://imputationserver.sph.umich.edu/share/results/eaed51016447957ed0cc3335e38c9b33/chr_8.zip
wget https://imputationserver.sph.umich.edu/share/results/6fbc45c47c5f96c820c92d45370ee8ae/chr_9.zip
wget https://imputationserver.sph.umich.edu/share/results/16841f72ec2009fb156f27988d8beab1/chr_X.no.auto_female.zip


# paste code above into bash script called download_data
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_down \
    --script_prefix ${ancestry}_impute_download \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_data_${ancestry}

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip \
        --script_prefix ${ancestry}_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done
rm -rf *zip

## AA
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.
* AA password: r4wBmEoSBQjl0I

Then, inflate imputation results.

In [ ]:
ancestry=aa
passW=r4wBmEoSBQjl0I
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209/final/$ancestry

# QC-results
wget https://imputationserver.sph.umich.edu/share/results/5c4609de8b0bfe335db6f0fdea7390ae/qcreport.html

# SNP Statistics
wget https://imputationserver.sph.umich.edu/share/results/e938672dbae111ec272269d44eebc2ab/statistics.txt

# Logs
wget https://imputationserver.sph.umich.edu/share/results/83918ccb9cd3eed6b537ce7c92ee0472/chr_1.log
wget https://imputationserver.sph.umich.edu/share/results/8d3d6f59b7891691b6bc08d68ef93033/chr_10.log
wget https://imputationserver.sph.umich.edu/share/results/c373cc129ca79d73df150a0ad2d0267b/chr_11.log
wget https://imputationserver.sph.umich.edu/share/results/a15d9833d1f1364513f074e52074ad7b/chr_12.log
wget https://imputationserver.sph.umich.edu/share/results/5de7b948fa6f40c32ebed56d843a52da/chr_13.log
wget https://imputationserver.sph.umich.edu/share/results/6eb431aebef6ae39b5a3c22c0b733c5c/chr_14.log
wget https://imputationserver.sph.umich.edu/share/results/2fc63cf0bc9951fc001383946cd67dee/chr_15.log
wget https://imputationserver.sph.umich.edu/share/results/504b5191ac485f47056006f6421e8c8f/chr_16.log
wget https://imputationserver.sph.umich.edu/share/results/9551021d261f347d9df918ca9e58a26e/chr_17.log
wget https://imputationserver.sph.umich.edu/share/results/f5c0f8ac41935d71fcf8f266d4260a78/chr_18.log
wget https://imputationserver.sph.umich.edu/share/results/a5534044f5b50a87c9a59ec8ea13caa3/chr_19.log
wget https://imputationserver.sph.umich.edu/share/results/f302c90156dc8eb0723f16baf8829db7/chr_2.log
wget https://imputationserver.sph.umich.edu/share/results/e61cd38ff99a385d95ff69841cfdfb6a/chr_20.log
wget https://imputationserver.sph.umich.edu/share/results/eada87d6ab38ea10b90500f9ad35bc60/chr_21.log
wget https://imputationserver.sph.umich.edu/share/results/2b37c829af5cf08e095f7606690b1947/chr_22.log
wget https://imputationserver.sph.umich.edu/share/results/938be86c1f8c7bc0a01a1fe9d2a341e4/chr_3.log
wget https://imputationserver.sph.umich.edu/share/results/62c99ebe9a50c309cec91f58259d0293/chr_4.log
wget https://imputationserver.sph.umich.edu/share/results/dd7e26d34b7397bcfdc99d310fed25b5/chr_5.log
wget https://imputationserver.sph.umich.edu/share/results/655d8edeae89ac6763143bf402a79b73/chr_6.log
wget https://imputationserver.sph.umich.edu/share/results/f7d5a744f11df6231b5fbd70ef8cc9ba/chr_7.log
wget https://imputationserver.sph.umich.edu/share/results/8f7701cd2bb5c0f1b6804c97071acca4/chr_8.log
wget https://imputationserver.sph.umich.edu/share/results/f1129f2a7aaad5a747fd28e82451ad35/chr_9.log
wget https://imputationserver.sph.umich.edu/share/results/f297e0b985514314a0b159a41dd105c6/chr_X.no.auto_female.log


# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/4b0775f9427d14106c0e9bf23b8cdc8e/chr_1.zip
wget https://imputationserver.sph.umich.edu/share/results/2974b00af98e6860f61e4aa12b0058c2/chr_10.zip
wget https://imputationserver.sph.umich.edu/share/results/710f8c91e5cfbc94b1663fd1013d1a3d/chr_11.zip
wget https://imputationserver.sph.umich.edu/share/results/dc41f8d0844b7c9939bec230336de10f/chr_12.zip
wget https://imputationserver.sph.umich.edu/share/results/c48bb4a998f56755bc4a3a11acbf10fd/chr_13.zip
wget https://imputationserver.sph.umich.edu/share/results/d51681d59e70e9e3fb6a0b6b8529c9f1/chr_14.zip
wget https://imputationserver.sph.umich.edu/share/results/50e201fcd58ef3dd95d953db6c6d7c8f/chr_15.zip
wget https://imputationserver.sph.umich.edu/share/results/613bd854dff6359f28d7385830ff66c4/chr_16.zip
wget https://imputationserver.sph.umich.edu/share/results/1e03c8709c65a8014e1e43dcb84fa44e/chr_17.zip
wget https://imputationserver.sph.umich.edu/share/results/7075737f88adfe878c8ff2406fae9c07/chr_18.zip
wget https://imputationserver.sph.umich.edu/share/results/829ed4ebbc920339ca7049008d80ccb8/chr_19.zip
wget https://imputationserver.sph.umich.edu/share/results/489ee092b7cda32d14d6871c6f3ab072/chr_2.zip
wget https://imputationserver.sph.umich.edu/share/results/c9360a34e1d22952ac7563b2f3ab78e8/chr_20.zip
wget https://imputationserver.sph.umich.edu/share/results/315d335d71a886e1427ab10fb1870941/chr_21.zip
wget https://imputationserver.sph.umich.edu/share/results/89c5712bc59fede952d62c6a3d45440e/chr_22.zip
wget https://imputationserver.sph.umich.edu/share/results/86490a9229e10b399b681183d9ec1191/chr_3.zip
wget https://imputationserver.sph.umich.edu/share/results/bdbe8ee95404295fbc9850f8adf4fcdf/chr_4.zip
wget https://imputationserver.sph.umich.edu/share/results/bc657969eb73e0007dd1012c095da531/chr_5.zip
wget https://imputationserver.sph.umich.edu/share/results/a85d0c08c4588a4c6736e3ba32acd50d/chr_6.zip
wget https://imputationserver.sph.umich.edu/share/results/d7d44badf0534d874357618d3f3ab2ef/chr_7.zip
wget https://imputationserver.sph.umich.edu/share/results/d829733d33af378421e8b6f4d381338f/chr_8.zip
wget https://imputationserver.sph.umich.edu/share/results/503e8a34a78b1a4e1bf9c9395e742038/chr_9.zip
wget https://imputationserver.sph.umich.edu/share/results/c23633892dbc9cdb05bbcc8ca54223b7/chr_X.no.auto_female.zip

# paste code above into bash script called download_data
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_down \
    --script_prefix ${ancestry}_impute_download \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_data_${ancestry}

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip \
        --script_prefix ${ancestry}_unzip \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done
## inflate chr results
#/shared/bioinformatics/software/scripts/qsub_job.sh \
#    --job_name ${ancestry}_unzip \
#    --script_prefix ${ancestry}_unzip \
#    --mem 6.8 \
#    --nslots 1 \
#    --priority 0 \
#    --program unzip -P $passW *zip 
# we can remove the original imputed data from MIS after we inflate the zip files
rm -rf *zip

# Upload to S3

In [ ]:
cd /shared/data/studies/heroin/uhs4/genotype/imputed/20181209

for ancestry in {aa,ea};do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_upload \
        --script_prefix ${ancestry}_upload \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program aws s3 sync ${ancestry} s3://rti-heroin/uhs4/data/genotype/imputed/${ancestry}/
done